In [43]:
from matplotlib import pyplot as plt
from pandas.tools.plotting import table
import subprocess
%matplotlib inline
import re
from functools import reduce
from tqdm import tqdm
import pandas as pd
import os
import collections
import scipy.stats
import numpy as np
import scipy
import csv
import math
from scipy import interp
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, auc
from sklearn import svm
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_recall_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
import subprocess
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import re
import seaborn as sns
from functools import reduce
from sklearn.model_selection import LeaveOneOut
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.utils.multiclass import unique_labels
import statsmodels.api as sm

In [44]:
dset=pd.read_csv ("clintab_GMQL/gencode.gene.info.v22.tsv", delimiter='\t')
gruppo=dset.groupby(['gene_type'])
proteingroup=gruppo.get_group('protein_coding')
lista1=set(proteingroup['gene_id'])
#len(proteingroup)

In [45]:
dset_rna_res=pd.read_csv("clintab_GMQL/gene_exprs/res/res_new.csv", delimiter='\t')
dset_rna_senl=pd.read_csv("clintab_GMQL/gene_exprs/sl_32/sl_new.csv", delimiter='\t')
dset_rna_sens=pd.read_csv("clintab_GMQL/gene_exprs/ss_32/ss_new.csv", delimiter='\t')

In [46]:
dset_res = dset_rna_res[dset_rna_res.ensemble_id.isin(lista1)]
protein_coding_res=pd.DataFrame(dset_res)
protein_coding_res['genes'] = protein_coding_res['ensemble_id']+'-'+ protein_coding_res['gene_symbol']
protein_coding_res.head()

,patient,chrom,start,stop,ensemble_id,entrez_id,gene_symbol,fpkm_uq,fpkm,genes
8,R_00000,chr1,69090,70008,ENSG00000186092.4,79501.0,OR4F5,0.000000,0.000000,ENSG00000186092.4-OR4F5
18,R_00000,chr1,182392,184158,ENSG00000279928.1,NaN,FO538757.3,0.000000,0.000000,ENSG00000279928.1-FO538757.3
19,R_00000,chr1,184922,200322,ENSG00000279457.2,NaN,FO538757.2,85509.848376,3.315533,ENSG00000279457.2-FO538757.2
29,R_00000,chr1,450739,451678,ENSG00000278566.1,729759.0,OR4F29,0.000000,0.000000,ENSG00000278566.1-OR4F29
49,R_00000,chr1,685715,686654,ENSG00000273547.1,81399.0,OR4F16,0.000000,0.000000,ENSG00000273547.1-OR4F16


In [47]:
dset_senl = dset_rna_senl[dset_rna_senl.ensemble_id.isin(lista1)]
protein_coding_senl=pd.DataFrame(dset_senl)
protein_coding_senl['genes'] = protein_coding_senl['ensemble_id']+'-'+ protein_coding_senl['gene_symbol']

In [48]:
dset_sens = dset_rna_sens[dset_rna_sens.ensemble_id.isin(lista1)]
protein_coding_sens=pd.DataFrame(dset_sens)
protein_coding_sens['genes'] = protein_coding_sens['ensemble_id']+'-'+ protein_coding_sens['gene_symbol']

In [49]:
resdf=protein_coding_res.pivot(index="patient", columns="genes", values="fpkm")
genes = resdf.columns.tolist()
#resdf.head()
resdf.shape

(60, 19814)

In [50]:
senldf=protein_coding_senl.pivot(index="patient", columns="genes", values="fpkm")
senldf = senldf[genes]
#senldf.head()
senldf.shape

(34, 19814)

In [51]:
sensdf=protein_coding_sens.pivot(index="patient", columns="genes", values="fpkm")
sensdf = sensdf[genes]
#sensdf.head()
sensdf.shape

(107, 19814)

In [76]:
resdf_median = resdf.median(0)
senldf_median = senldf.median(0)
sensdf_median = sensdf.median(0)

In [77]:
resdf_median.shape

(19814,)

In [78]:
print(len(resdf_median), len(senldf_median), len(sensdf_median))

19814 19814 19814


In [79]:
pvalues = []
for gene in tqdm(genes):
    try:
        gene_pvalue = scipy.stats.mannwhitneyu(sensdf[gene],resdf[gene]).pvalue
    except Exception:
        gene_pvalue = 1
    pvalues.append(gene_pvalue)
       
pvalues = pd.DataFrame({
    "gene": genes,
    "resistant_median": resdf_median.values,
    #"sensitive_long_median": senldf_median.values,
    "sensitive_short_median": sensdf_median.values,
    "mannwhiteney_pvalue": pvalues
})

pvalues.head()

100%|██████████| 19814/19814 [00:08<00:00, 2264.31it/s]


,gene,resistant_median,sensitive_short_median,mannwhiteney_pvalue
0,ENSG00000000003.13-TSPAN6,15.537437,15.924936,0.362456
1,ENSG00000000005.5-TNMD,0.038791,0.067985,0.023603
2,ENSG00000000419.11-DPM1,42.847159,43.144585,0.329259
3,ENSG00000000457.12-SCYL3,1.939755,2.014650,0.203584
4,ENSG00000000460.15-C1orf112,1.415162,1.640842,0.044616


## Bonferroni con len(pvalues)

In [32]:
p_value_corr =[]

for index, value in tqdm(pvalues['gene'].iteritems()): 
        
        somma=len(pvalues)
        p_value=pvalues[pvalues['gene']==value]['mannwhiteney_pvalue'].iloc[0]
        p_value_corr.append(somma*p_value)
        #print(p_value_corr)
print(len(p_value_corr))

19814it [00:55, 355.69it/s]

19814


In [33]:
id_name=[]
gene_name=[]

for x in ["-".join(x.strip().split("-")[:1])  for x in (pvalues['gene'])]:
    id_name.append(x)   
for x in ["-".join(x.strip().split("-")[1:])  for x in (pvalues['gene'])]:
    gene_name.append(x) 
    
pvalues['ensemble_id']=id_name
pvalues['gene_symbol']=gene_name
pvalues=pd.DataFrame(pvalues)

In [34]:
pvalues['p_value_corr']=p_value_corr
#pvalues.head()

In [35]:
threshold = 0.05

significant_genes_corr = pvalues[(pvalues.p_value_corr < threshold)].copy()
significant_genes_corr = significant_genes_corr.sort_values("mannwhiteney_pvalue", ascending=True)
print(significant_genes_corr.shape[0])

1


In [36]:
significant_genes_corr.head()

,gene,sensitive_long_median,sensitive_short_median,mannwhiteney_pvalue,ensemble_id,gene_symbol,p_value_corr
10886,ENSG00000163554.10-SPTA1,0.0,0.00689,0.000002,ENSG00000163554.10,SPTA1,0.046536


## Bonferroni con len(num_patients)

In [84]:
threshold = 0.0005

significant_genes = pvalues[(pvalues.mannwhiteney_pvalue < threshold)].copy()
significant_genes = significant_genes.sort_values("mannwhiteney_pvalue", ascending=True)
print(significant_genes.shape[0])

58


In [85]:
id_name=[]
gene_name=[]

for x in ["-".join(x.strip().split("-")[:1])  for x in (significant_genes['gene'])]:
    id_name.append(x)   
for x in ["-".join(x.strip().split("-")[1:])  for x in (significant_genes['gene'])]:
    gene_name.append(x) 
    
significant_genes['ensemble_id']=id_name
significant_genes['gene_symbol']=gene_name
significant_genes=pd.DataFrame(significant_genes)

In [86]:
significant_genes.head()

,gene,resistant_median,sensitive_short_median,mannwhiteney_pvalue,ensemble_id,gene_symbol
6158,ENSG00000130255.11-RPL36,311.832565,183.251963,0.000006,ENSG00000130255.11,RPL36
14675,ENSG00000182253.13-SYNM,0.305599,0.510293,0.000017,ENSG00000182253.13,SYNM
1887,ENSG00000090861.14-AARS,20.941002,27.409527,0.000019,ENSG00000090861.14,AARS
18171,ENSG00000233927.4-RPS28,314.732910,197.475554,0.000021,ENSG00000233927.4,RPS28
603,ENSG00000044574.7-HSPA5,107.221321,142.558129,0.000041,ENSG00000044574.7,HSPA5


In [87]:
#significant_genes.to_csv('new_data/gene_exprs/r_sl_pv2.csv', sep=',', header=True, index=False)
significant_genes.to_csv('new_data/gene_exprs/r_ss_pv2.csv', sep=',', header=True, index=False)
#significant_genes.to_csv('new_data/gene_exprs/sl_ss_pv2.csv', sep=',', header=True, index=False)

In [ ]:
significant_genes_corr.head()

In [ ]:
listageni=significant_genes_corr['gene'].values
#listageni

## Plot

In [ ]:
for g in listageni:

    boxplot1=pd.DataFrame(resdf, columns=[g])
    boxplot2=pd.DataFrame(senldf, columns=[g])
    fig, ax= plt.subplots()
    boxplot3=pd.concat([boxplot1, boxplot2], axis=1)
    ax.set_title('Res vs Senl gene: '+ g, fontsize = 10)

    boxplot_tot=boxplot3.boxplot(fontsize = 10)
    
    #print(boxplot3)
    #plt.savefig('new_data/gene_exprs/boxplot/res_sl/boxplot{}.png'.format(g))

In [ ]:
sns.set(style="whitegrid", font_scale=1.5)

for g in listageni[:]:
    data = pd.DataFrame(columns=["Values", "Class"])
    for row in resdf[g]:
        data = data.append({"Values":row,"Class":"Res"}, ignore_index=True)
    for row in senldf[g]:
        data = data.append({"Values":row,"Class":"SenL"}, ignore_index=True)
    
    plt.figure()
    sns.violinplot(data=data, x="Class", y="Values", palette="muted", saturation=0.75, alpha=0.7)
    plt.title(g)
    #plt.savefig('new_data/gene_exprs/violinplot/res_sl/violinplot{}.png'.format(g))

## Standard deviation

In [ ]:
r_std=resdf.std()
sl_std=senldf.std()
ss_std=sensdf.std()

#print (r_std)

In [ ]:
x=r_std.sort_values(axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
y=ss_std.sort_values(axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
z=sl_std.sort_values(axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [ ]:
res_std=x.to_frame()
res_std['genes']=res_std.index
res_std.index=np.arange(len(res_std))
#res_std

In [ ]:
sens_std=y.to_frame()
sens_std['genes']=sens_std.index
sens_std.index=np.arange(len(sens_std))
#sens_std

In [ ]:
senl_std=z.to_frame()
senl_std['genes']=senl_std.index
senl_std.index=np.arange(len(senl_std))
#senl_std

In [ ]:
threshold = 100.00

significant_std =x[(x > threshold)].copy()
significant_std = significant_std.sort_values(ascending=True)
print(significant_std.shape[0])

In [ ]:
w=significant_std.to_frame()

In [ ]:
w['genes']=w.index
w.index=np.arange(len(w))
#w

In [ ]:
#controllare quali sono i geni che risultano salvi tra boxplot, std, correzioni da test multiplo (qua
#basterebbe anche solo Bonferroni ma se poi me ne salva troppo pochi è tutt'un niente), fare 
#l'intersezione per definire le features, creare un nuovo dset contenente:
#solo le features, e la loro posizione (chrom, start, stop da dare a Sara)

In [ ]:
lista1=set(w['genes'])
lista2=set(significant_genes_corr ['gene'])

In [ ]:
inters=lista1.intersection(lista2)

In [ ]:
inters

## Appartenenza geni-classe

In [ ]:
gene_sl_ss=pd.read_csv("new_data/gene_exprs/sl_ss_bonf_ge.csv", delimiter=',')
gene_r_ss=pd.read_csv("new_data/gene_exprs/r_ss_bonf_ge.csv", delimiter=',')
gene_r_sl=pd.read_csv("new_data/gene_exprs/r_sl_bonf_ge.csv", delimiter=',')

In [ ]:
gene_sl_ss.head()

In [ ]:
lista3=set(gene_sl_ss['gene'])
lista4=set(gene_r_ss['gene'])
lista5=set(gene_r_sl['gene'])

In [ ]:
inters_=lista3.intersection(lista4)

In [ ]:
inters_

## Benjamini-Hochbergh

In [85]:
p_sorted=pvalues.sort_values(by='mannwhiteney_pvalue')

In [86]:
a=p_sorted['mannwhiteney_pvalue'].get_values()

In [87]:
enumerate(a)

In [88]:
num_total_tests=len(pvalues)

In [89]:
def calc_benjamini_hochberg_corrections(p_values, num_total_tests):
   
    p_value_bh = []
    for i, p_value in enumerate(p_values):
        bh_value = p_value * num_total_tests / (i + 1)
        p_value_bh.append(bh_value)
    return (p_value_bh)

In [90]:
p_value_corr_bh=calc_benjamini_hochberg_corrections(a, num_total_tests)

In [91]:
p_sorted['p_value_corr_bh']=p_value_corr_bh

In [92]:
p_sorted.head()

,gene,resistant_median,sensitive_long_median,mannwhiteney_pvalue,p_value_corr_bh
1511,ENSG00000081026.17-MAGI3,2.004508,3.137310,0.000053,1.054464
7361,ENSG00000137393.9-RNF144B,8.484803,15.532160,0.000065,0.639372
6820,ENSG00000134504.11-KCTD1,6.369336,10.695657,0.000086,0.566977
9336,ENSG00000151812.13-SLC35F4,0.005034,0.015873,0.000091,0.452034
7359,ENSG00000137364.4-TPMT,10.070143,13.108867,0.000117,0.464476


In [93]:
threshold = 0.05

significant_genes_bh = p_sorted[(p_sorted.p_value_corr_bh < threshold)].copy()
significant_genes_bh = significant_genes_bh.sort_values("p_value_corr_bh", ascending=True)
print(significant_genes_bh.shape[0])

0


In [50]:
id_name=[]
gene_name=[]

for x in ["-".join(x.strip().split("-")[:1])  for x in (significant_genes_bh['gene'])]:
    id_name.append(x)   
for x in ["-".join(x.strip().split("-")[1:])  for x in (significant_genes_bh['gene'])]:
    gene_name.append(x) 
    
significant_genes_bh['ensemble_id']=id_name
significant_genes_bh['gene_symbol']=gene_name
significant_genes_bh=pd.DataFrame(significant_genes_bh)

In [51]:
significant_genes_bh

,gene,sensitive_long_median,sensitive_short_median,mannwhiteney_pvalue,p_value_corr_bh,ensemble_id,gene_symbol
10886,ENSG00000163554.10-SPTA1,0.0,0.00689,0.000002,0.046536,ENSG00000163554.10,SPTA1


In [ ]:
#significant_genes_bh.to_csv('new_data/gene_exprs/sl_ss_bh_ge.csv', sep=',', header=True, index=False)

## FDR other method

In [52]:
from statsmodels.stats.multitest import fdrcorrection

In [53]:
pvalues_FDR=pvalues['mannwhiteney_pvalue']
p_corrected=fdrcorrection(pvalues_FDR, alpha=0.05, method='indep')
w=p_corrected[1]

p_sorted['p_value_corr_bh']=w
p_sorted.head()

,gene,sensitive_long_median,sensitive_short_median,mannwhiteney_pvalue,p_value_corr_bh
10886,ENSG00000163554.10-SPTA1,0.000000,0.006890,0.000002,0.345944
12805,ENSG00000171311.11-EXOSC1,10.670756,7.311312,0.000011,0.473731
4750,ENSG00000118242.14-MREG,8.046079,5.166310,0.000011,0.501786
8382,ENSG00000143919.13-CAMKMT,3.003575,1.978113,0.000018,0.489024
2870,ENSG00000104290.9-FZD3,5.618396,3.202372,0.000028,0.501030


In [54]:
threshold = 0.05

significant_genes_bh = p_sorted[(p_sorted.p_value_corr_bh < threshold)].copy()
significant_genes_bh = significant_genes_bh.sort_values("p_value_corr_bh", ascending=True)
print(significant_genes_bh.shape[0])

1
